## Jack's car Rental problem. Stupid_Policy defines the policy for original problem for taking actions in moving cars from one location to another. Lady_policy defines the updated Jack's car rental problem of moving the cars each night ##

In [5]:
#%%
import numpy as np
import pdb
import itertools

In [6]:
#%%poisson distribution
prob_space = {}
def poisson(x,lam):
    key = tuple(list((x,lam)))
    if key not in prob_space.keys():
        prob_space[key] = np.exp(-lam)*lam**x/np.math.factorial(x)
    return prob_space[key]

action_choice = np.arange(-5,6)

## Iterated over 2 policies ##

In [7]:
#%%jack environment
class car_Rental():
    def __init__(self,location,action):
        self.location = location
        self.action_space = np.arange(action)
        self.num_state = len(location)
        self.max_req = 11
        self.max_ret = 11
    
    def reward(self,action,state_dict):
        tot_reward = 0
        for req_A in range(self.max_req):
            for req_B in range(self.max_req):
                prob_req = poisson(req_A,3)*poisson(req_B,4)
                #update number of cars on both locations
                car_A = min(req_A,self.location[0])
                car_B = min(req_B,self.location[1])
                #current reward
                reward = 10*(car_A+car_B)
                #final location of cars
                car_A = min(max(self.location[0] - car_A - action, 0),20)
                car_B = min(max(self.location[1] - car_B - action, 0),20)
                
                reward = reward + (-2*np.abs(action))
                
                for ret_A in range(self.max_ret):
                    for ret_B in range(self.max_ret):
                        prob_ret = poisson(ret_A,3) * poisson(ret_B,2)
                        prob_joint = prob_req*prob_ret
                        car_A = min((car_A + ret_A),20)
                        car_B = min((car_B +ret_B),20)
                        reward = prob_joint*(reward + \
                                             gamma*(state[tuple(list((car_A,car_B)))]))
                        tot_reward = tot_reward + reward
        return tot_reward
    
    def stupid_policy(self,action,state):
        reward_action = []
        for i in action:
            tot_reward = 0
            prob_store = []
            for req_A in range(self.max_req):
                for req_B in range(self.max_req):
                    prob_req = poisson(req_A,3)*poisson(req_B,4)
                    car_A = min(req_A,self.location[0])
                    car_B = min(req_B,self.location[1])
                    reward = 10*(car_A+car_B)
                    car_A = min(max(self.location[0] - car_A - i, 0),20)
                    car_B = min(max(self.location[1] - car_B - i,0),20)
                    reward = reward + (-2*np.abs(i))
                    
                    for ret_A in range(self.max_ret):
                        for ret_B in range(self.max_ret):
                            prob_ret = poisson(ret_A,3) * poisson(ret_B,2)
                            prob_joint = prob_req*prob_ret
                            car_A = min((car_A + ret_A),20)
                            car_B = min((car_A + ret_B),20)
                            prob_store.append(prob_joint)
                            reward = prob_joint*(reward + \
                                                 gamma*(state[tuple(list((car_A,car_B)))]))
                            tot_reward  = tot_reward + reward
            reward_action.append(tot_reward)
        return action[np.argmax(reward_action)]
    
    def lady_policy(self,action,state):
        reward_action = []
        for i in action:
            tot_reward = 0
            prob_store = []
            for req_A in range(self.max_req):
                for req_B in range(self.max_req):
                    prob_req = poisson(req_A,3) * poisson(req_B,4)
                    car_A = min(req_A,self.location[0])
                    car_B = min(req_B,self.location[1])
                    reward = 10*(car_A+car_B)
                    car_A = min(max(self.location[0] - car_A - i , 0), 20)
                    car_B = min(max(self.location[1] - car_B - i, 0),20)
                    reward = reward + (-2*np.abs(i))
                    
                    for ret_A in range(self.max_ret):
                        for ret_B in range(self.max_ret):
                            prob_ret = poisson(ret_A,3) * poisson(ret_B,2)
                            prob_joint = prob_req * prob_ret
                            j = [1,-1]
                            for k in range(2):
                                car_A = car_A + ret_A + j[k]
                                car_B = car_B + ret_B - j[k]
                                if car_A > 20 and car_A < 30:
                                    car_A = 20
                                else:
                                    car_A = min((car_A + ret_A),20)
                                    
                                if car_B > 20 and car_B < 30:
                                    car_B = 20
                                else:
                                    car_B = min((car_A + ret_A),20)
                                prob_store.append(prob_joint)
                                reward = reward + 4
                                reward = prob_joint*(reward + \
                                                     gamma*(state[tuple(list((car_A,car_B)))]))
                                tot_reward = tot_reward + reward
            reward_action.append(tot_reward)
        return action[np.argmax(reward_action)]

In [8]:
#%%policy improvement and evaluation
action_all = np.arange(-5,6)

gamma = 0.9

theta = 0.9

loop = True

init_state = list(itertools.product(np.arange(21),repeat = 2))

state = {}

for i in range(len(init_state)):
    state[init_state[i]] = 20
    
policy = {}

for i in range(len(init_state)):
    policy[init_state[i]] = 0

policy_stable = False

while policy_stable == False:
    while loop == True:
        delta = 0.0
        for i in range(len(init_state)):
            action_choice = policy[init_state[i]]
            
            v = state[init_state[i]]
            
            agent_car = car_Rental(init_state[i],20)
            
            value_next = agent_car.reward(action_choice,state)
            
            delta = max(delta,np.abs((v-value_next)))
            
            state[init_state[i]] = value_next
            
        if delta<theta:
            loop = False
            
    for i in range(len(init_state)):
        old_action = policy[init_state[i]]
        
        agent_car = car_Rental(init_state[i],20)
        
        #pi_policy = agent_car.stupid_policy(action_all,state)
        pi_policy = agent_car.lady_policy(action_all,state)
        
        if old_action == pi_policy:
            policy_stable = True
        else:
            loop == True
            policy[init_state[i]] = pi_policy